In [3]:
# Binary Logistic Regression
# Using the Titanic dataset 

"""
MODEL SUMMARY: 
Loss function: Cross Entropy
Hidden function: Sigmoid
Output function: Sigmoid

USER INFORMATION:
User can specify the neural network using Structure
Structure -> [input_dim, hidden layer_dims, ..., ouput_dim]
"""

import numpy as np
import pandas as pd

"""
A neural network using the sigmoid activation function and 
logisitic regression loss function.
"""
class Neural_Network:
    def __init__(self, structure=[7,12,1], lr=0.1, epochs=100, up_freq=10):
        
        np.random.seed(0)
        self.units = structure
        
        self.lr = lr
        self.num_iterations = epochs
        self.update_freq = up_freq
        
        self.weights = []  
        self.biases = []
        
        self.Z = []
        self.A = []
        
        # add a specified number of units based on structure       
        for i in range(len(self.units) - 1):            
            weight = np.random.rand(self.units[i + 1], self.units[i])
            self.weights.append(weight)            
            bias = np.zeros((self.units[i + 1], 1))
            self.biases.append(bias)       
    
    """
    Basic sigmoid function
    """
    def sigmoid(self, X):
        return 1/(1 + np.exp(-X))    
    
    """
    Compute the cost using the logistic regression cost function.
    """
    def compute_cost(self, Y):
        m = Y.shape[1]        
        cost_sum = np.multiply(np.log(self.A[-1]), Y) +  np.multiply((1 - Y), np.log(1 - self.A[-1]))
        cost = np.squeeze(-np.sum(cost_sum) / m)        
        return cost
    
    """
    Perform a forward prediction with the model.
    """    
    def feed_forward(self, X):
        
        self.A = []
        self.Z = []
        
        z1 = np.dot(self.weights[0], X) + self.biases[0]
        a = self.sigmoid(z1)
        self.Z.append(z1)
        self.A.append(a)
        
        for i in range(len(self.units) - 2):
            z = np.dot(self.weights[i+1], a) + self.biases[i+1]
            a = self.sigmoid(z)            
            self.Z.append(z)
            self.A.append(a) 
            
    """
    Back propagate through the learned network.
    """    
    def back_prop(self, X, Y):
        
        m = Y.shape[1]
        
        # compute derivative of cost and sigmoid
        dz = self.A[-1] - Y
        dw = (1/m) * np.dot(dz, self.A[-2].T)
        db = (1/m) * np.sum(dz, axis=1, keepdims=True)
        self.weights[-1] = self.weights[-1] - self.lr * dw
        self.biases[-1] = self.biases[-1] - self.lr * db
          
        # cycle through hidden layers of NN 
        for i in range(len(self.units) - 3, 0, -1):
            dz = np.dot(self.weights[i+1].T, dz) * self.sigmoid(self.A[i]) * (1 - self.sigmoid(self.A[i]))
            dw = (1/m) * np.dot(dz, self.A[i - 1].T)
            db = (1/m) * np.sum(dz, axis=1, keepdims=True)
            self.weights[i] = self.weights[i] - self.lr * dw
            self.biases[i] = self.biases[i] - self.lr * db
        
        # compute output
        dz = np.dot(self.weights[1].T, dz) * self.sigmoid(self.A[0]) * (1 - self.sigmoid(self.A[0]))         
        dw = (1/m) * np.dot(dz, X.T)
        db = (1/m) * np.sum(dz, axis=1, keepdims=True)
        
        self.weights[0] = self.weights[0] - self.lr * dw
        self.biases[0] = self.biases[0] - self.lr * db
    
    """
    Train the model on a sample of date.
    """    
    def train(self, X, Y):
        
        for i in range(1, self.num_iterations + 1):
            self.feed_forward(X)  
            
            if i % self.update_freq == 0: 
                
                Y_pred = np.around(self.A[-1], 0).astype(int)                  
                
                print('Epoch: {} Cost: {}'.format(i, round(self.compute_cost(Y), 2)))
                print('Accuracy: {}'.format(round((1 - np.abs(np.sum(Y - Y_pred)/Y.shape[1])) * 100, 2)))                
                
            self.back_prop(X, Y)
    
    """
    Test the learned model against the true values.
    """           
    def test(self, X, Y_true):
        self.feed_forward(X)          
        Y_pred = np.around(self.A[-1], 0).astype(int) 
        
        print('#######################')
        print('Test Accuracy: {}'.format(round((1 - np.abs(np.sum(Y_pred - Y_true)/Y_true.shape[1])) * 100, 2)))                


"""
Convert the data into a usable form and then split the 
sample into training and testing.
"""
def pre_process(df):
    
    # remove any unhelpful columns
    del df['Name']
    del df['Ticket']
    del df['Cabin']
    
    # convert object columns to int or float columns
    df['Sex'] = df['Sex'].eq('male').mul(1)
    df['Embarked'] = pd.factorize(df['Embarked'])[0] + 1    
    df = df.dropna()   
    df = df.sample(frac=1)
    
    # convert to array    
    y = np.array([df['Survived'].to_numpy()])
    x = df.loc[:, df.columns != 'Survived'].to_numpy().T
    
    # split into test and training
    y_train = y[:, 50:]
    x_train = x[:, 50:]
    y_test = y[:, :50]
    x_test = x[:, :50]
        
    return y_train, x_train, y_test, x_test        
    
if __name__ == "__main__":
    
    # load and process the data
    orig_train = pd.read_csv('./Data/titanic_train.csv', index_col=0)
    train = orig_train.copy()       
    y_train, x_train, y_test, x_test = pre_process(train)
    
    # initialse the network
    network = Neural_Network([7, 8, 12, 1],
                             lr=0.05,
                             epochs=500,
                             up_freq=100
                            )
        
    # train and test
    network.train(x_train, y_train)
    network.test(x_test, y_test)    
    

Epoch: 100 Cost: 0.67
Accuracy: 65.51
Epoch: 200 Cost: 0.65
Accuracy: 84.49
Epoch: 300 Cost: 0.64
Accuracy: 90.21
Epoch: 400 Cost: 0.64
Accuracy: 93.67
Epoch: 500 Cost: 0.63
Accuracy: 95.78
#######################
Test Accuracy: 92.0
